<a href="https://colab.research.google.com/github/Marvin2798/CNN-Project-/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Link : https://github.com/ironhack-labs/project-1-deep-learning-image-classification-with-cnn/blob/main/README.md#project-i--deep-learning-image-classification-with-cnn

In [1]:
from tensorflow.keras.datasets import cifar10
#loading the dataset since we can access it through keras
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [3]:
#normalise the pixel values to 0 and 1 by dividing by 255
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
